In [5]:
import django
import os
import json
os.environ['DJANGO_SETTINGS_MODULE'] = 'django_settings'
django.setup()

In [6]:
from boto3.session import Session
import boto3
import botocore
import pandas as pd

In [7]:
from loader import load_credential

In [8]:
ACCESS_KEY = load_credential("AWS_ACCESS_KEY_ID","")
SECRET_ACCESS_KEY = load_credential("AWS_SECRET_ACCESS_KEY","")

In [9]:
from data_management.models import *

In [ ]:
################# download image from csv: 사용할 csv와 다운받을 버킷을 지정해 줘야 사용 가능합니다. #################

In [7]:
class S3Manager():
    def __init__(self):
        self.access_key = load_credential("AWS_ACCESS_KEY_ID","")
        self.secret_access_key = load_credential("AWS_SECRET_ACCESS_KEY","")
        self.s3 =  boto3.resource('s3',
                                aws_access_key_id=ACCESS_KEY,
                                aws_secret_access_key=SECRET_ACCESS_KEY)

    def get_file_lists(self, bucket_name, s3_folder):
        lists = []
        if not s3_folder:
            for s3_file in self.s3.Bucket(bucket_name).objects.all():
                lists.append(s3_file.key)
            return lists
        for s3_file in self.s3.Bucket(bucket_name).objects.all():
            if s3_file.key.split('/')[0] == s3_folder:
                lists.append(s3_file.key.split('/')[1])
        return lists
    
    def extract_training_lists(self, csv_file, file_list):
        df = pd.read_csv(csv_file)
        csv_list = []
        new_file_list = []
        for i in range(len(df)):
            csv_list.append(df['origin_url'][i].split('/')[-1])
        for i in range(len(file_list)):
            for j in range(len(csv_list)):
                if(file_list[i] == csv_list[j]):
                    new_file_list.append(file_list[i])
        return new_file_list
    
    def download_s3_files(self, bucket_name, s3_folder, csv_file, local_path):
        # local_path 는 jupyter를 실행한 폴더 기준
        # 폴더 경로는 \\로 끝나야 함 (for Window)
        # TODO : pandas로 csv 읽고 filelist에 있는 것만 남기기
        file_list = self.get_file_lists(bucket_name, s3_folder)
        file_list = self.extract_training_lists(csv_file, file_list)
        i = 0
        for file in file_list:
            i += 1
            print(i)
            if s3_folder:
                KEY = s3_folder + '/' + file
            else:
                KEY = file
            print(KEY)
            try:
                self.s3.Bucket(bucket_name).download_file(KEY, local_path + file)
                print('downloaded')
            except botocore.exceptions.ClientError as e:
                if e.response['Error']['Code'] == "404":
                    print("The object does not exist.")
                else:
                    print('----')
                    raise
        print('DONE!')

In [8]:
# download from monde-data : BucketNmae --> 크롤링 파일들 박싱한 데이터 저장한 버킷
# s3manager.download_s3_files('버킷이름', ''폴더이름, 'csv파일', '저장경로')

In [9]:
s3manager1 = S3Manager()

In [ ]:
s3manager1.download_s3_files('monde-data', 'original-bag-images-dev', './crop_download_test.csv', './data/')

In [11]:
# download from temp-originalimage : BucketNmae --> hdf5파일들 저장한 버킷
# s3manager.download_s3_files('버킷이름', ''폴더이름, 'csv파일', '저장경로')
# 폴더가 없으므로 비워둠

In [12]:
s3manager2 = S3Manager()

In [ ]:
s3manager2.download_s3_files('temp-originalimage', '', './s3upload_download_test.csv', './data/')

In [ ]:
################# download image from queryset : bucket은 temp-originalimage & monde-data 두개에서 다운 받습니다. #################
# TODO : mondebro에서 태깅한 데이터 사용 가능하게 

In [127]:
class S3Manager2():
        def __init__(self):
            self.access_key = load_credential("AWS_ACCESS_KEY_ID","")
            self.secret_access_key = load_credential("AWS_SECRET_ACCESS_KEY","")
            self.s3 =  boto3.resource('s3',
                                    aws_access_key_id=ACCESS_KEY,
                                    aws_secret_access_key=SECRET_ACCESS_KEY)
            self.temp_bucket_list = self.get_s3_file_temp_bucket()
            self.mondedata_bucket_list = self.get_s3_file_mondedata_bucket()
            
        
        def get_s3_file_temp_bucket(self):
            lists = []
            for s3_file in self.s3.Bucket('temp-originalimage').objects.all():
                lists.append(s3_file.key)
            return lists
        
        def get_s3_file_mondedata_bucket(self):
            lists = []
            for s3_file in self.s3.Bucket('monde-data').objects.all():
                if s3_file.key.split('/')[0] == 'original-bag-images-dev':
                    lists.append(s3_file.key)
            return lists
        
        # s3_image_url 이 존재할 때
        def get_list_from_queryset(self, queryset, n):
            new_file_list_t = []
            new_file_list_m = []
            
            qs_temp_bucket = queryset.filter(s3_image_url__isnull=False)
            temp_bucket_list = []
            qs_mondedata_bucket = queryset.exclude(image="")
            mondedata_bucket_list = []
            
            for q in qs_temp_bucket:
                name = q.s3_image_url.split('/')[-1]
                temp_bucket_list.append(name)
                
            for q in qs_mondedata_bucket:
                name = q.image.name
                mondedata_bucket_list.append(name)
            
            pre_temp_bucket_list = self.temp_bucket_list
            
            #temp_bucket
            for i in range(len(pre_temp_bucket_list)):
                for j in range(len(temp_bucket_list)):
                    if(pre_temp_bucket_list[i] == temp_bucket_list[j]):
                        new_file_list_t.append(pre_temp_bucket_list[i])
        
            pre_mondedata_bucket_list = self.mondedata_bucket_list
            
            #mondedata_bucket
            for i in range(len(pre_mondedata_bucket_list)):
                for j in range(len(mondedata_bucket_list)):
                    if(pre_mondedata_bucket_list[i] == mondedata_bucket_list[j]):
                        new_file_list_m.append(pre_mondedata_bucket_list[i])
                        
            if n == 1: # temp bucket
                return new_file_list_t
            elif n == 2:
                return new_file_list_m
            else:
                print('ERROR')
    
        def download_s3_files_from_queryset(self, local_path, queryset):
            # local_path 는 jupyter를 실행한 폴더 기준
            # 폴더 경로는 \\로 끝나야 함 (for Window)
            file_list_t = self.get_list_from_queryset(queryset, 1)
            file_list_m = self.get_list_from_queryset(queryset, 2)
            i = 0
            for file in file_list_t:
                i += 1
                KEY = file
                dest_pathname = os.path.join(local_path, KEY)
                if i % 100 == 0:
                    print(i)
                    print(KEY)
                try:
                    self.s3.Bucket('temp-originalimage').download_file(KEY, dest_pathname)
                except botocore.exceptions.ClientError as e:
                    if e.response['Error']['Code'] == "404":
                        print("The object does not exist.")
                    else:
                        print('----')
                        raise
            print('======')
        
            i = 0
            for file in file_list_m:
                i += 1
                KEY = file
                path_key = KEY.split('/')[1]
                dest_pathname = os.path.join(local_path, path_key)
                if i % 100 == 0:
                    print(i)
                    print(KEY)
                try:
                    self.s3.Bucket('monde-data').download_file(KEY, dest_pathname)
                except botocore.exceptions.ClientError as e:
                    if e.response['Error']['Code'] == "404":
                        print("The object does not exist.")
                    else:
                        print('----')
                        raise
            print('DONE')

In [19]:
CroppedImage.objects.filter(origin_source__image_review=True).last().origin_source.s3_image_url

'https://temp-originalimage.s3.ap-northeast-2.amazonaws.com/square_24_XPJO3185PYE_1.jpg'

In [54]:
OriginalImage.objects.filter(image_review=True).count()

1700

In [128]:
a=S3Manager2()

In [129]:
queryset = OriginalImage.objects.filter(image_review=True)

In [ ]:
a.download_s3_files_from_queryset('./data/', queryset)